<div style="font-size: 200%; font-weight: bold; color: maroon;">Red de contactos</div>
<!-- Paulo Villegas, 2020 - v. 1.0 -->

Una red de contactos (contact network) es un tipo de grafo de personas usado en epidemiología, en el que los enlaces representan personas que han entrado en contacto y por tanto han podido suponer una vía de difusión.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import networkx as nx

%matplotlib inline
plt.style.use('bmh')
mpl.rcParams['figure.figsize'] = (16, 10)

# Datos

Usamos un dataset real: el publicado en:

> Rossana Mastrandrea, Julie Fournet, Alain Barrat, [_Contact Patterns in a High School: A Comparison between Data Collected Using Wearable Sensors, Contact Diaries and Friendship Surveys_](https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0136497), 2015
   
Contiene una red de contactos medida en un instituto francés, siguiendo a los estudiantes de varios cursos, durante unos cuantos días. Cada enlace supone el momento en que dos personas entraron en contacto.

Lee datos de ficheros CSV a DataFrames de Pandas

In [ ]:
contacts = pd.read_csv('DATA/High-School_data_2013.csv.gz', delimiter=' ', 
                       names=('time', 'id1', 'id2', 'cat1', 'cat2'), header=None)

In [ ]:
len(contacts)

In [ ]:
contacts['timestamp'] = pd.to_datetime(contacts.time, unit='s')

In [ ]:
contacts.head()

In [ ]:
sorted(contacts.cat1.unique())

# Red

Crea la red. En este caso es un multigrafo, porque enter dos personas puede haber más de un enlace (si a lo largo de este tiempo estuvieron en contacto más de una vez)

In [ ]:
# Crea la red desde el DataFrame de enlaces. Aparte de origen & destino, añadimos la marca de tiempo
cnetwork = nx.from_pandas_edgelist(contacts,'id1','id2', ['time'], create_using=nx.MultiGraph)

# Añadimos atributos de los nodos (categoría de persona)
for r in contacts.itertuples(index=False):
    cnetwork.add_node(r.id1, cat=r.cat1)
    cnetwork.add_node(r.id2, cat=r.cat2)

In [ ]:
len(cnetwork)

In [ ]:
len(cnetwork.edges)

In [ ]:
len(cnetwork.nodes)

In [ ]:
nx.diameter(cnetwork)

# Grado

Calculamos el grado de cada nodo

In [ ]:
dg = nx.degree(cnetwork)

Vamos a representar el grado de cada nodo, ordenado de mayor a menor grado. La estructura original de `dg` es una lista de tuplas; lo movemos a un diccionario ordenado (por grado)

In [ ]:
from collections import OrderedDict
from operator import itemgetter

# Crea el diccionario de grados
dg_ord = OrderedDict( ((k,v) for k, v in sorted(iter(dg), key=itemgetter(1), reverse=True)))

In [ ]:
# Pintamos un diagrama de barras
fig, ax = plt.subplots(figsize=(16,10))
ax.bar(range(len(dg_ord)), dg_ord.values(), align="center")

ax.set_xlim(0, len(dg_ord));
#ax.xaxis.set_ticks(np.arange(0, len(dg_ord), 10))
ax.set_xticklabels(list(dg_ord.keys())[::70], fontdict={'rotation': 'vertical'});
ax.set_ylabel("grado del nodo");
ax.set_xlabel("nodeId");

# Representación de la red

In [ ]:
nodelab = { i: cnetwork.nodes[i]['cat'] for i in cnetwork}
set(nodelab.values())

In [ ]:
%%time

nx.draw_kamada_kawai(cnetwork, node_size=620, with_labels=True, labels=nodelab, 
                     edge_color="gray", font_color='yellow')